In [ ]:
import xgboost as xgb
import numpy as np
import sys
import os
from sklearn import model_selection
from sklearn.datasets import dump_svmlight_file

sys.path.append(r'D:\UCI\CS273A-ML\HW4-code')
import mltools as ml

data_path = r'D:\UCI\CS273A-ML\Project'
X = np.genfromtxt(os.path.join(data_path, 'X_train.txt'), delimiter=None)
Y = np.genfromtxt(os.path.join(data_path, 'Y_train.txt'), delimiter=None)
Xte = np.genfromtxt(os.path.join(data_path, 'X_test.txt'), delimiter=None)
# wts = np.genfromtxt('HW4-code\weights.txt', delimiter=None)

X,Y = ml.shuffleData(X,Y)
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y)

In [ ]:
print(Xtr.shape, Xva.shape)

In [ ]:
# subsample = 100000
Xt, Yt = Xtr[:], Ytr[:] # subsample for efficiency (you can go higher)
Xv, Yv = Xva[:], Yva[:] # subsample for efficiency (you can go higher)

## Choose train and eval

In [ ]:
x_train = Xt
y_train = Yt
x_val = Xv
y_val = Yv
x_test = Xte

In [ ]:
# dump_svmlight_file(x_train, y_train, os.path.join(data_path, 'dtrain.svm'), zero_based=True)
# dump_svmlight_file(x_val, y_val, os.path.join(data_path, 'dval.svm'), zero_based=True)

# dtrain = xgb.DMatrix('dtrain.svm')
# dval = xgb.DMatrix('dval.svm')
dtrain = xgb.DMatrix(x_train, label=y_train)
dval = xgb.DMatrix(x_val, label=y_val)

In [ ]:
param = {'max_depth': 7, 'min_child_weight':2 ,'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(dtrain, 'train'), (dval, 'eval')]
num_round = 5000
early_stopping_rounds=10

In [ ]:
bst = xgb.train(param, dtrain, nu6m_round, evallist, early_stopping_rounds=early_stopping_rounds)
bst.dump_model('dump.raw.txt')

In [ ]:
dtest = xgb.DMatrix(Xte)
print(bst.best_score, bst.best_iteration, bst.best_ntree_limit)
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
Yte = np.vstack((np.arange(Xte.shape[0]), ypred)).T

In [ ]:

np.savetxt(os.path.join(data_path, 'Y_submit_xgboosted_best_iter_d7.txt'), Yte, '%d, %.2f', header='ID,Prob1', comments='', delimiter=',')

In [ ]:
Yte[:,1]

In [ ]:
y_ = bst.predict(dtrain)

[2189]	train-auc:0.796306	eval-auc:0.753815 - Xt, Yt (max-depth=3)
train-auc:0.796306	eval-auc:0.753815 - Xt_norm, Yt (max-depth=3)
train-auc:	eval-auc: - Xt_norm, Yt (max-depth=6)
train-auc:0.873119	eval-auc:0.780095 (.7 t-v split, max-depth=6) - NOT SUBMITTED
[986]	train-auc:0.899646	eval-auc:0.797286 (MAX-DEPTH=7)
